In [2]:
from sys import exit
import json
import deepl
import os
import re

def load_settings():
    global master_data, translator
    # デフォルト値

    master_data = {
        "auth_key": "",
        "source_lang": "EN",
        "target_lang": "JA"
    }
    try:
        with open("config.json") as fp:
            update_data = json.load(fp)
        for k, v in update_data.items():
            master_data[k] = v
    except Exception as e:
        pass



def translate(source_string):
    # Get Replacement rule
    #rules_path = f"rules/{master_data['source_lang']}_{master_data['target_lang']}.txt"
    #replacement_rule = []
    #if os.path.exists(rules_path):
    #    with open(rules_path, "r") as fp:
    #        rules_raw = fp.readlines()
    #    for rule in rules_raw:
    #        tmp_split = rule.replace("\n", "").split("\t")
    #        if(len(tmp_split)) >= 2:
    #            replacement_rule.append([tmp_split[0], tmp_split[1]])

    # split contents
    sources, tmp = [], []
    cnt = 0
    for s in source_string.splitlines(keepends=True):
        cnt += len(s)
        if cnt > 4500:
            sources.append("".join(tmp))
            tmp = [s]
            cnt = len(s)
        else:
            tmp.append(s)
    else:
        sources.append("".join(tmp))

    # DeepL Translate
    try:
        translator = deepl.Translator(auth_key=master_data["auth_key"])
        result = translator.translate_text(
                    [sources], 
                    source_lang=master_data["source_lang"],
                    target_lang=master_data["target_lang"],
                    tag_handling="xml",
                    ignore_tags=["keep","keep2"])
        usage = translator.get_usage()
        result_txt = "\n".join([r.text for r in result])
    except Exception as ex:
        return "", "DeepL translation error\n"+str(ex)

    # ルールベースで置き換え
    message = ""
    #for rule in replacement_rule:
    #    try:
    #        #print(rule)
    #        result_txt = re.sub(rule[0], rule[1], result_txt)
    #    except Exception as ex:
    #        message = f"Replacement rule error at {rule[0]} -> {rule[1]}\n"+str(ex)
    #
    if message == "":
        return result_txt, f"Translate success\nCharacter usage: {usage.character.count} of {usage.character.limit}"
    else:
        return result_txt, message

In [13]:
load_settings()

In [5]:
# stringの読み込み
source = "../sample/test.md"
output_dir = "../sample/output"

lines = []
load_str = ''
with open(f"{source}", encoding="utf-8") as f:
    lines = f.readlines()    # mdファイルの内容を1行ずつ配列に入れる
load_str = ''.join(lines)    # 全行を取得後、配列を結合して文字列変数に入れ

In [6]:
# $aa$びように挟まれた場所を翻訳されないように、<keep>aaa</keep>で挟む。
# いいやり方を思いつかなかったので、交互に変換する。ちゃんと交互になっていると信じる。
for i in range(10000):
    load_str = load_str.replace(r"$",r"<keep>",1)
    load_str = load_str.replace(r"$",r"</keep>",1)

In [14]:
# $$aa$$の場合は、<keep></keep>になってしまっているので、そこは<keep2>aaa</keep2>に変える。
for i in range(1000):
    load_str = load_str.replace(r"<keep></keep>",r"<keep2>",1)
    load_str = load_str.replace(r"<keep></keep>",r"</keep2>",1)

In [15]:
load_str

'# Workload-Aware Anonymization\n\nKristen LeFevre <keep>^{1} \\quad</keep> David J. DeWitt <keep>{ }^{1} \\quad</keep> Raghu Ramakrishnan <keep>{ }^{1,2}</keep><br><keep>{ }^{1}</keep> University of Wisconsin - Madison, 1210 West Dayton St., Madison, WI 53706<br><keep>{ }^{2}</keep> Yahoo! Research, 701 First Ave., Sunnyvale, CA 94089\n\n\n#### Abstract\n\nProtecting data privacy is an important problem in microdata distribution. Anonymization algorithms typically aim to protect individual privacy, with minimal impact on the quality of the resulting data. While the bulk of previous work has measured quality through one-size-fits-all measures, we argue that quality is best judged with respect to the workload for which the data will ultimately be used.\n\nThis paper provides a suite of anonymization algorithms that produce an anonymous view based on a target class of workloads, consisting of one or more data mining tasks, as well as selection predicates. An extensive experimental evalua

In [16]:
with open(f"{output_dir}/middle.md", mode="w") as f:
    f.write(load_str)

In [17]:
result_txt, message = translate(load_str)

In [29]:
from copy import deepcopy
result_ = deepcopy(result_txt)

In [30]:
result_

'# ワークロードを考慮した匿名化\n\nKristen LeFevre<keep>^{1} \\quad</keep> David J. DeWitt<keep>{ }^{1} \\quad</keep> Raghu Ramakrishnan<keep>{ }^{1,2}</keep><br> <keep>{ }^{1}</keep> University of Wisconsin - Madison, 1210 West Dayton St., Madison, WI 53706<br><keep>{ }^{2}</keep> Yahoo! Research, 701 First Ave., Sunnyvale, CA 94089\n\n\n#### 要旨\n\nデータのプライバシー保護は、マイクロデータの配布において重要な問題である。匿名化アルゴリズムは通常、個人のプライバシーを保護し、結果として得られるデータの品質への影響を最小限に抑えることを目的としている。これまでの研究の大部分は、一律に品質を測定してきたが、我々は、品質とは、データが最終的に使用される作業負荷に関して最もよく判断されるものであると主張する。\n\n本論文では、1つまたは複数のデータマイニングタスクと選択述語からなる作業負荷のターゲットクラスに基づいて匿名化ビューを生成する一連の匿名化アルゴリズムを提供します。広範な実験的評価により、このアプローチは、これまでの匿名化技術よりもしばしば効果的であることが示された。\n\n\n### 単一ターゲット分類モデル\n\n多次元再符号化を用いた匿名化のために、Mondrianアルゴリズムが最近提案された[17]。このアルゴリズムは、（多次元）準識別子領域空間の貪欲な再帰的分割に基づく（図3参照）。ほぼ均一なパーティション占有率を得るために、[17]は、正規化された値の範囲が最大の分割属性を再帰的に選択し、（連続または順序属性の場合）分割属性の中央値を中心にデータをパーティション化することを提案している。このプロセスは、許容される分割がなくなるまで繰り返される。つまり、匿名性の制約や値の汎化階層によって課される制約に違反しない限り、特定の領域をさらに分割することができないことを意味している。このアルゴリズムをMedian Mondrianと呼ぶ。\n

In [31]:
result_ = re.sub(r"<keep>",r" \$\$", result_)
result_ = re.sub(r"</keep>", r"\$ ",result_)
result_ = re.sub(r"<keep2>", r"$$\n\\begin{align*}", result_)
result_ = re.sub(r"</keep2>", r"\\end{align*}\n$$", result_)

In [32]:
result_

'# ワークロードを考慮した匿名化\n\nKristen LeFevre \\$\\$^{1} \\quad\\$  David J. DeWitt \\$\\${ }^{1} \\quad\\$  Raghu Ramakrishnan \\$\\${ }^{1,2}\\$ <br>  \\$\\${ }^{1}\\$  University of Wisconsin - Madison, 1210 West Dayton St., Madison, WI 53706<br> \\$\\${ }^{2}\\$  Yahoo! Research, 701 First Ave., Sunnyvale, CA 94089\n\n\n#### 要旨\n\nデータのプライバシー保護は、マイクロデータの配布において重要な問題である。匿名化アルゴリズムは通常、個人のプライバシーを保護し、結果として得られるデータの品質への影響を最小限に抑えることを目的としている。これまでの研究の大部分は、一律に品質を測定してきたが、我々は、品質とは、データが最終的に使用される作業負荷に関して最もよく判断されるものであると主張する。\n\n本論文では、1つまたは複数のデータマイニングタスクと選択述語からなる作業負荷のターゲットクラスに基づいて匿名化ビューを生成する一連の匿名化アルゴリズムを提供します。広範な実験的評価により、このアプローチは、これまでの匿名化技術よりもしばしば効果的であることが示された。\n\n\n### 単一ターゲット分類モデル\n\n多次元再符号化を用いた匿名化のために、Mondrianアルゴリズムが最近提案された[17]。このアルゴリズムは、（多次元）準識別子領域空間の貪欲な再帰的分割に基づく（図3参照）。ほぼ均一なパーティション占有率を得るために、[17]は、正規化された値の範囲が最大の分割属性を再帰的に選択し、（連続または順序属性の場合）分割属性の中央値を中心にデータをパーティション化することを提案している。このプロセスは、許容される分割がなくなるまで繰り返される。つまり、匿名性の制約や値の汎化階層によって課される制約に違反しない限り、特定の領域をさらに分割することができないことを意味している。このアルゴリズムをMedian Mondrianと呼ぶ。\n\nターゲットマイニ

In [33]:
with open(f"{output_dir}/result2.md", mode="w") as f:
    f.write(result_)